# WEB SCRAPING ASSIGNMENT 3

In [1]:
#Import Libraries
import pandas as pd
import time
import selenium

# Importing selenium webdriver 
from selenium import webdriver

#Importing requests
import requests

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing ActionChains for hovering action
from selenium.webdriver.common.action_chains import ActionChains

### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [160]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the homepage of Amazon.in
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(2)

search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar

# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : Guitar


In [161]:
search_bar.send_keys(user_inp)                                   # sending user input to search bar
driver.find_element_by_id("nav-search-submit-button").click()    # clicking the search button to start the search
time.sleep(4)

### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [162]:
# scrape all product urls
product_urls = []
start=0
end=3
for page in range(start,end):#for loop for scrapping 3 page
    url=driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')       #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href"))                        #appending the urls in product_urls list
    nxt_button=driver.find_element_by_xpath("//li[@class='a-last']//a")     #scraping the list of buttons from the page
    nxt_button.click()             # clicking on next button
    time.sleep(2)

In [164]:
#Make empty lists
brand=[]
name=[]
ratings=[]
no_ratings=[]
price=[]
returns=[]
expected_delivery=[]
availability=[]
other_details=[]

In [170]:
for url in product_urls: # running for loop over each extracted URL to scrape data
    driver.get(url) # opening the URL page
    time.sleep(2)
    
    # Extracting brand name from xpath
    try:
        b=driver.find_element_by_xpath("//table[@class='a-normal a-spacing-micro']/tbody/tr[1]/td[2]")
        brand.append(b.text)
    except NoSuchElementException:
        brand.append("-")
        
    # Extracting name from id
    try:
        n=driver.find_element_by_id("productTitle")
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
    
    # Extracting ratings from 
    try:
        r=driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']//span")
        ratings.append(r.text)
    except NoSuchElementException:
        ratings.append("-")
        
    # Extracting No of ratings from xpath
    try:
        nr=driver.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']/div/span[3]/a/span")
        no_ratings.append(nr.text.split(" ")[0])
    except NoSuchElementException:
        no_ratings.append("-")
        
    # Extracting Price from xpath
    try:
        p=driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
        price.append(p.text)
    except NoSuchElementException:
        price.append("-")
        
    # Extracting Returns from xpath
    try:
        rt=driver.find_element_by_xpath("//div[@class='a-column a-span3 a-text-center icon-container icon-grid-width celwidget']/span/div[2]/a")
        returns.append(rt.text)
    except NoSuchElementException:
        returns.append("-")
        
    # Extracting Expected Delivery from xpath
    try:
        ed=driver.find_element_by_xpath("//div[@id='dynamicDeliveryMessage']/div/b")
        expected_delivery.append(ed.text)
    except NoSuchElementException:
        expected_delivery.append("-")
        
    # Extracting Availability from xpath
    try:
        a=driver.find_element_by_xpath("//div[@id='availability']/span")
        availability.append(a.text)
    except NoSuchElementException:
        availability.append("-")
        
    # Extracting Other Details from 
    try:
        od=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        other_details.append(od.text)
    except NoSuchElementException:
        other_details.append("-")
        


In [172]:
# Make dataframe for the product details
product = pd.DataFrame({'Brand':brand,'Name':name,'Rating':ratings,'No. of ratings':no_ratings,'Price':price,
                        'Return/Exchange':returns,'Expected Delivery':expected_delivery,'Availability':availability,
                        'Other Details':other_details,'URL':product_urls})
# printing the first rows of the dataframe
product.head(5)   

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Medellin,Medellin MED-BLU-C Linden Wood Acoustic Guitar,3.9 out of 5,516,"₹2,399.00",7 Days Replacement,"Wednesday, Aug 25",In stock.,Material: Wood\nColour: Blue\nAcoustic Guitar,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5,980,-,7 Days Replacement,"Wednesday, Aug 25",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Kadence,"Kadence Frontier Series, Natural Acoustic Guit...",3.9 out of 5,74,-,7 Days Replacement,"Wednesday, Aug 25",In stock.,"Product Type : Acoustic Guitar\nBrand:Kadence,...",https://www.amazon.in/gp/slredirect/picassoRed...
3,Cort,Cort AD810 Dreadnought Acoustic Guitar with Gi...,4.4 out of 5,136,"₹10,250.00",7 Days Replacement,"Sunday, Aug 22",In stock.,The Cort AD810 Dreadnought Acoustic Guitar has...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"12,124","₹2,649.00",7 Days Replacement,"Sunday, Aug 22",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [142]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the google images
url = "https://images.google.com/"
driver.get(url)

In [147]:
#Creating empty lists to store the image src
fruits_img=[]
cars_img=[]
ML_img=[]

In [144]:
#finding the search bar element tag and entering "fruits" in search bar
search=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search.send_keys("fruits")
#Clicking the search button
driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").click()
time.sleep(2)

# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(600):
    driver.execute_script("window.scrollBy(0,10000)")

In [148]:
# Scraping the URLs of the Fruit Images
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    source=i.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            fruits_img.append(source)

In [150]:
driver.find_element_by_id("REsRA").clear() #Clearing the search bar
driver.find_element_by_id("REsRA").send_keys("cars") #Entering "cars" in the search bar
driver.find_element_by_xpath("//span[@class='n6h3Rc']").click() #Clicking search button
time.sleep(2)

# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(600):
    driver.execute_script("window.scrollBy(0,10000)")

In [151]:
# Scraping the URLs of the Car Images
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    source=i.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            cars_img.append(source)

In [154]:
driver.find_element_by_id("REsRA").clear() #Clearing the search bar
driver.find_element_by_id("REsRA").send_keys("Machine Learning") #Entering "Machine Learning" in the search bar
driver.find_element_by_xpath("//span[@class='n6h3Rc']").click() #Clicking search button
time.sleep(2)

# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(600):
    driver.execute_script("window.scrollBy(0,10000)")

In [155]:
# Scraping the URLs of the Machine Learning Images
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    source=i.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            ML_img.append(source)

In [157]:
# Downloading the Images
for i in range(100):
    
    #Fruits
    response= requests.get(fruits_img[i])
    file = open(r"E:\DataTrained\Images\Fruits"+str(i)+".jpg", "wb")
    file.write(response.content)
    
    # Cars
    response= requests.get(cars_img[i])
    file = open(r"E:\DataTrained\Images\Cars"+str(i)+".jpg", "wb")
    file.write(response.content)
    
    # Machine Learning
    response= requests.get(ML_img[i])
    file = open(r"E:\DataTrained\Images\MachineLearning"+str(i)+".jpg", "wb")
    file.write(response.content)

In [158]:
file.close() # closing the file

In [159]:
# creating Dataframe of the scraped urls of the images
images=pd.DataFrame({"Fruits":fruits_img[:100],
                    "Cars":cars_img[:100],
                    "Machine Learning":ML_img[:100]})

# printing the DataFrame
images

,Fruits,Cars,Machine Learning
0,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
98,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...


### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [55]:
#Connecting the webdriver
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(3)

# Opening the www.flipkart.com
url ="https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

try:
    login_X=driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']") # Button to close the login popup
    login_X.click()
except NoSuchElementException:
    print("No Login Page")

In [56]:
search_bar=driver.find_element_by_xpath("//input[@title='Search for products, brands and more']") # Finding the search bar using its xpath
search_bar.clear()           # Clearing the Search bar
search_bar.send_keys("pixel")     # Inputing the keyword to search

search_button=driver.find_element_by_xpath("//button[@class='L0Z3Pu']") # Finding the search button using its xpath
search_button.click()        # Clicking the Search Button

In [57]:
# creating empty lists for scraping the data
brand=[]
smartphone=[]
color=[]
RAM=[]
ROM=[]
pri_cam=[]
sec_cam=[]
display_size=[]
display_res=[]
processor=[]
processor_core=[]
batt_capacity=[]
batt_type=[]
price=[]
pro_urls=[]

In [58]:
# Fetching urls of phones coming on 1st page
for i in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    pro_urls.append(i.get_attribute('href'))

In [59]:
# Scraping data from each url
for url in pro_urls:
    driver.get(url) # Loading the webpage URL
    
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click() # Button for expanding the spec
    
    # Extracting Brand name from xpath
    try:
        b=driver.find_element_by_xpath("//span[@class='B_NuCI']")
        brand.append(b.text.split()[0])
    except NoSuchElementException:
        brand.append("-")
        
    # Extracting Smartphone name from xpath
    try:
        s=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]/ul/li")
        smartphone.append(s.text)
    except NoSuchElementException:
        smartphone.append("-")
        
    # Extracting Color from xpath 
    try:
        c=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]/ul/li")
        color.append(c.text)
    except NoSuchElementException:
        color.append("-")
        
    # Extracting RAM from xpath
    try:
        ra=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[1]/td[2]/ul/li")
        RAM.append(ra.text)
    except NoSuchElementException:
        RAM.append("-")
        
    # Extracting ROM from xpath 
    try:
        ro=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[2]/td[2]/ul/li")
        ROM.append(ro.text)
    except NoSuchElementException:
        ROM.append("-")
        
    # Extracting Primary Camera from xpath
    try:
        pm=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[2]/td[2]/ul/li")
        pri_cam.append(pm.text)
    except NoSuchElementException:
        pri_cam.append("-")
        
    # Extracting Secondary Camera from xpath
    try:                                                                                    
        sec_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sc = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sc = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        sec_cam.append(sc.text)
    except NoSuchElementException:
        sec_cam.append('-')
    
    # Extracting Display Size from xpath 
    try:
        ds=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[1]/td[2]")
        display_size.append(ds.text)
    except NoSuchElementException:
        display_size.append("-")
        
    # Extracting Display Resolution from xpath
    try:
        dr=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[2]/td[2]")
        display_res.append(dr.text)
    except NoSuchElementException:
        display_res.append("-")
        
    # Extracting Processor from xpath
    try:
        pro=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[1]")
        if pro.text!="Processor Type" : raise NoSuchElementException
        p=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[2]/ul/li")
        processor.append(p.text)
    except NoSuchElementException:
        processor.append("-")
        
    # Extracting Processor Core from xpath 
    try:
        cores=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[3]/td[1]")
        if cores.text!="Processor Core" : raise NoSuchElementException
        c=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[3]/td[2]/ul/li")
        processor_core.append(c.text)
    except NoSuchElementException:
        processor_core.append("-")
        
    # Extracting Battery Capacity from xpath 
    try:
        if driver.find_element_by_xpath("//div[@class='_3k-BhJ'][10]/div").text!="Battery & Power Features":
            if driver.find_element_by_xpath("//div[@class='_3k-BhJ'][9]/div").text=="Battery & Power Features":
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[2]/ul/li")
            elif driver.find_element_by_xpath("//div[@class='_3k-BhJ'][8]/div").text=="Battery & Power Features":
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[1]/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap= driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[1]/td[2]/ul/li')             
        batt_capacity.append(bat_cap.text)
    except NoSuchElementException:
        batt_capacity.append('-')
        
    # Extracting Battery Type from xpath 
    try:
        if driver.find_element_by_xpath("//div[@class='_3k-BhJ'][10]/div").text!="Battery & Power Features":
            if driver.find_element_by_xpath("//div[@class='_3k-BhJ'][9]/div").text=="Battery & Power Features":
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ=driver.find_element_by_xpath("//div[@class='_3k-BhJ'][9]/table[1]/tbody/tr[2]/td[2]/ul/li")
            elif driver.find_element_by_xpath("//div[@class='_3k-BhJ'][8]/div").text=="Battery & Power Features":
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')             
        batt_type.append(bat_typ.text)
    except NoSuchElementException:
        batt_type.append('-')
        
    # Extracting Price from xpath
    try:
        p= driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')    
        price.append(p.text)
    except NoSuchElementException:
        price.append('-')

In [61]:
#creating Dataframe for the scraped data of the phones
phone=pd.DataFrame({"Brand":brand,
                   "Smartphone":smartphone,
                   "Colour":color,
                   "RAM":RAM,
                   "Storage(ROM)":ROM,
                   "Primary Camera":pri_cam,
                   "Secondary Camera":sec_cam,
                   "Display Size":display_size,
                   "Display Resolution":display_res,
                   "Processor":processor,
                   "Processor Core":processor_core,
                   "Battery Capacity":batt_capacity,
                   "Battery Type":batt_type,
                   "Price":price,
                   "Product URL":pro_urls})
# printing dataframe
phone

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Core,Battery Capacity,Battery Type,Price,Product URL
0,Google,Pixel 4a,Just Black,128 GB,6 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,-,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Google,Pixel 3a,Clearly White,64 GB,4 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3000 mAh,-,"₹39,999",https://www.flipkart.com/google-pixel-3a-clear...
2,Google,Pixel 3,Just Black,64 GB,4 GB,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,13.97 cm (5.5 inch),2160 x 1080 pixels,Qualcomm Snapdragon 845 64-bit,Octa Core,2915 mAh,-,"₹65,999",https://www.flipkart.com/google-pixel-3-just-b...
3,REDMI,Note 9,Pebble Grey,64 GB,4 GB,48MP + 8MP + 2MP + 2MP,-,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,Lithium Polymer,"₹11,999",https://www.flipkart.com/redmi-note-9-pebble-g...
4,REDMI,Note 9,Shadow Black,64 GB,4 GB,48MP + 8MP + 2MP + 2MP,-,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,Lithium Polymer,"₹11,999",https://www.flipkart.com/redmi-note-9-shadow-b...
5,REDMI,Note 9,Aqua Green,64 GB,4 GB,48MP + 8MP + 2MP + 2MP,-,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,Lithium Polymer,"₹11,999",https://www.flipkart.com/redmi-note-9-aqua-gre...
6,Redmi,Redmi 9,Sporty Orange,64 GB,4 GB,13MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,-,5000 mAh,Li-Po,"₹9,899",https://www.flipkart.com/redmi-9-sporty-orange...
7,Redmi,Redmi 9,Sky Blue,64 GB,4 GB,13MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,-,5000 mAh,Li-Po,"₹9,789",https://www.flipkart.com/redmi-9-sky-blue-64-g...
8,KARBONN,KX3,"Black,Red",32 MB,32 MB,0.3MP Rear Camera,-,4.57 cm (1.8 inch),128 x 160$$pixel,-,-,800 mAh,Li-ion,₹878,https://www.flipkart.com/karbonn-kx3/p/itm60cf...
9,Redmi,Redmi 9A,Nature Green,32 GB,2 GB,13MP Rear Camera,-,16.59 cm (6.53 inch),720 x 1600$$pixel,-,-,5000 mAh,-,"₹7,447",https://www.flipkart.com/redmi-9a-nature-green...


Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [20]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening google maps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

In [21]:
search=driver.find_element_by_id("searchboxinput")            # locating search bar
search.clear()                                                # clearing search bar
time.sleep(2)
search.send_keys(input('Enter City:'))                        # entering values in search bar
button=driver.find_element_by_id("searchbox-searchbutton")    # locating search button
button.click()                                                # clicking search button
time.sleep(3)

Enter City:Mumbai


In [22]:
#Fetching URL of current page
url=driver.current_url
#Splitting the URL string to extract Latitude and Longitude
lat_lng=str(url.split("@")[1:][0]).split(",")
#Printing URL,latitude and longitude
print("URL Extracted:",url)          # printing URL
print("Latitude:",str(lat_lng[0]))   # printing Latitude
print("Longitude:",str(lat_lng[1]))  # printing Longitude

URL Extracted: https://www.google.co.in/maps/place/Mumbai,+Maharashtra/@19.0820391,72.6009774,10z/data=!3m1!4b1!4m5!3m4!1s0x3be7c6306644edc1:0x5da4ed8f8d648c69!8m2!3d19.0759837!4d72.8776559
Latitude: 19.0820391
Longitude: 72.6009774


### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [85]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening trak.in
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

In [86]:
funding_deal=driver.find_element_by_xpath("//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(funding_deal)

In [87]:
#Creating empty lists for the required data
date=[]
startup=[]
industry=[]
sub_vertical=[]
city=[]
investor=[]
investment_type=[]
amount=[]

In [88]:
for i in range(48,51):
    # Date
    dt=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[2]".format(i))
    for d in dt:
        date.append(d.text)
        
    # Startup Name
    stup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in stup:
        startup.append(n.text)
        
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        industry.append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        sub_vertical.append(s.text)
        
    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for c in loc:
        city.append(c.text)
        
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        investor.append(n.text)
        
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        investment_type.append(n.text)
        
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        amount.append(a.text)
        
    try:
        # checking if there is any Next Page is there
        if driver.find_element_by_xpath("//a[@id='tablepress-{}_next']".format(i)).get_attribute('class')=="paginate_button next":
            driver.find_element_by_xpath("//a[@id='tablepress-{}_next']".format(i)).click() #clickig on Next button
            
            # Fetching data from next page
            # Date
            dt=driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[2]".format(i))
            for d in dt:
                date.append(d.text)
                
            # Startup Name
            stup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
            for n in stup:
                startup.append(n.text)
        
            # Industry/Vertical
            ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
            for n in ind:
                industry.append(n.text)
    
            # Sub-Vertical
            sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
            for s in sv:
                sub_vertical.append(s.text)
        
            # Location
            loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
            for c in loc:
                city.append(c.text)
        
            # Investor
            inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
            for n in inv:
                investor.append(n.text)
        
            # Investment Type
            invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
            for n in invt:
                investment_type.append(n.text)
        
            # Amount
            amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
            for a in amt:
                amount.append(a.text)
        
    except:
        continue # if no next page then continue


In [90]:
# creating dataframe for the scraped data of Funding Deals
funding_deal=pd.DataFrame({})
funding_deal['Date'] = date
funding_deal['Startup Name'] = startup
funding_deal['Industry/Vertical'] = industry
funding_deal['Sub-Vertical'] = sub_vertical
funding_deal['City/Location'] = city
funding_deal['Investor'] = investor
funding_deal['Investment Type'] = investment_type
funding_deal['Amount(in USD)'] = amount

# printing DataFrame
funding_deal

,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


### 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [91]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening www.digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [93]:
#clicking on top 10 option 
top_10=driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/ul/li[4]/a")
top_10.click()
time.sleep(2)

In [95]:
#clicking on laptops option
laptops=driver.find_element_by_xpath("//div[@class='categoty_list']/button[2]")
laptops.click()
time.sleep(2)

In [96]:
# getting best gaming laptops link and opening the page
best_gaming=driver.find_element_by_xpath("//div[@id='laptops']//div[3]//a")
driver.get(best_gaming.get_attribute('href'))
time.sleep(2)

In [97]:
# creating empty lists for scraping data
name=[]
price=[]
OS=[]
display=[]
processor=[]
memory=[]
weight=[]
dimension=[]
graphics_pro=[]

In [98]:
# scraping names
for i in driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3"):
    name.append(i.text)

# scraping price
for i in driver.find_elements_by_xpath("//table[@id='summtable']/tbody/tr/td[3]"):
    price.append(i.text)
    
# scraping OS
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]"):
    OS.append(i.text)
    
# scraping display
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]"):
    display.append(i.text)
    
# scraping processor
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]"):
    processor.append(i.text)
    
# scraping memory
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]"):
    memory.append(i.text)
    
# scraping weight
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]"):
    weight.append(i.text)
    
# scraping dimensions
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]"):
    dimension.append(i.text)
    
# scraping graphics_pro
for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]"):
    graphics_pro.append(i.text)

In [100]:
# creating dataframe for the scraped data of laptop
laptops=pd.DataFrame({"Name":name,
                "Operating System":OS,
                "Display":display,
                "Memory":memory,
                "Processor":processor,
                "Weight":weight,
                "Dimension":dimension,
                "Graphics processor":graphics_pro,
                "Price":price})

# printing dataframe
laptops

,Name,Operating System,Display,Memory,Processor,Weight,Dimension,Graphics processor,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,10th Generation Intel® Core™ i7-10700 | 2.90 GHz,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,N/A
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",1 TB SSD/16 GBGB DDR4,10th Generation Intel® Core™ i9-10980HK | NA,NA,NA,NA,₹341990
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,AMD Ryzen™ 9 5900HX | 3.3 GHz,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,N/A
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,AMD 3rd Generation Ryzen 9 | 3.3 GHz,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,₹164990
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,10th Generation Intel® Core™ i5-10300H | 2.50 GHz,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,₹76988
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",512 GB SSD/4 GBGB DDR4,Intel Core i7 10th Gen 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,₹185000
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",512 GB SSD/8 GBGB DDR4,AMD Ryzen™ 5-5500U hexa-core | NA,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,₹64370


### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [21]:
#set chromodriver.exe path
driver=webdriver.Chrome('chromedriver.exe')

In [22]:
#launch URL
url="https://www.forbes.com"
driver.get(url)

In [23]:
#clicking on the Menu hamburger
driver.find_element_by_xpath("//button[@class='icon--hamburger']").click()

In [24]:
#object of ActionChains
a=ActionChains(driver)
#Identifying element to hover
b=driver.find_element_by_xpath("//li[@class='header__channel header__color--centennial-silver header__hoverable']")
#hover over element
a.move_to_element(b).perform()
#clicking on "World'd Billionaires" from the sub-menu list
driver.find_element_by_xpath("//ul[@class='header__sections header__color--centennial-silver']//li[2]//a").click()

In [25]:
#creating empty lists for scraping data
rank=[]
name=[]
net_worth=[]
age=[]
citizenship=[]
source=[]
industry=[]

In [26]:
#Scraping Rank
for i in driver.find_elements_by_xpath("//div[@class='rank']"):
    rank.append(i.text.split(".")[0])
    
#Scraping Name
for i in driver.find_elements_by_xpath("//div[@class='personName']"):
    name.append(i.text)
    
#Scraping Net Worth
for i in driver.find_elements_by_xpath("//div[@class='netWorth']"):
    net_worth.append(i.text)
    
#Scraping Age
for i in driver.find_elements_by_xpath("//div[@class='age']"):
    age.append(i.text)
    
#Scraping Citizenship
for i in driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']"):
    citizenship.append(i.text)
    
#Sraping Source
for i in driver.find_elements_by_xpath("//div[@class='source']"):
    source.append(i.text)
    
#Scraping Industry
for i in driver.find_elements_by_xpath("//div[@class='category']"):
    industry.append(i.text)

In [28]:
#checking the length of each list
print(len(rank),len(name),len(net_worth),len(age),len(citizenship),len(source),len(industry))

200 200 200 200 200 200 200


In [29]:
#creating dataframe for the scraped data of billionaires
df=pd.DataFrame({"Rank":rank,
                "Name":name,
                "Net Worth":net_worth,
                "Age":age,
                "Citizenship":citizenship,
                 "Source":source,
                "Industry":industry})
#printing the data
df

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [101]:
#set chromodriver.exe path
driver=webdriver.Chrome('chromedriver.exe')

url="https://www.youtube.com/watch?v=cGc_NfiTxng"
driver.get(url)
time.sleep(4)

In [ ]:
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

In [111]:
# creating empty lists for scraping data
comments=[]
likes=[]
time=[]

In [112]:
# Scraping the comments
for i in driver.find_elements_by_xpath("//yt-formatted-string[@id='content-text']"):
    comments.append(i.text)

# Scraping the comment upvotes
for i in driver.find_elements_by_xpath("//span[@id='vote-count-middle']"):
    likes.append(i.text)
    
# Scraping the Time when the comment was posted
for i in driver.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text above-comment style-scope ytd-comment-renderer']"):
    time.append(i.text)

In [114]:
# creating the dataframe for the scraped data of YouTube Video comments
youtube=pd.DataFrame({"Comments":comments[:500],
                     "Comment Upvote":likes[:500],
                     "Time when comments was posted":time[:500]})

# Printing the DataFrame
youtube

,Comments,Comment Upvote,Time when comments was posted
0,I just realized this was the first kpop song I...,1K,5 months ago (edited)
1,Fun fact: This man didn't knew he was making a...,585,2 months ago
2,Fun fact: this was our childhood,370,3 months ago
3,"Don’t lie, this is the best K-Pop song.",112,3 weeks ago
4,i love how PSY knew that he wanted this to be ...,345,5 months ago
...,...,...,...
495,GME go fuckin BRrrrrrrrr!,3,6 days ago
496,Everybody be Gangnam style until the North Kor...,2,5 months ago (edited)
497,2021 pp's this is lit,,3 weeks ago
498,The reddit?......She is Kaput!?......,6,2 days ago


### 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [25]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe")

# opening https://www.hostelworld.com/
url="https://www.hostelworld.com/"
driver.get(url)
time.sleep(3)

In [26]:
driver.find_element_by_id("location-text-input-field").click()            # Clicking on the Search bar "Where do you want to go"
time.sleep(1)
driver.find_element_by_id("search-input-field").send_keys("London")       # Entering the location name "London"
time.sleep(3)
driver.find_element_by_xpath("//ul[@id='predicted-search-results']//li[2]//div").click() # Selecting "London,England" the first option from drop-down
time.sleep(3)

In [27]:
driver.find_element_by_id("search-button").click() # clicking on the "Let's go!" button

In [29]:
# Creating empty lists for scraping data
hostel_name=[]
distance=[]
ratings=[]
total_reviews=[]
overall_reviews=[]
privates_from_price=[]
dorms_from_price=[]
facilities=[]
facility=[]
property_desc=[]
urls=[]
prices=[]

In [30]:
for i in range(3):
    # Scraping Hostel names and their URLs
    for j in driver.find_elements_by_xpath("//h2[@class='title title-6']//a"):
        hostel_name.append(j.text)             #  hostel name
        urls.append(j.get_attribute('href'))   # URLs
    
    # Scraping Distance from City Centre
    for j in driver.find_elements_by_xpath("//span[@class='description']"):
        distance.append(j.text)
        
    # Scraping Prices which includes both Privates and Dorms
    for j in driver.find_elements_by_xpath("//div[@class='price-col']"):
        prices.append(j.text)
     
    try:
        # checking if this element was present
        if driver.find_element_by_xpath("//div[@class='pagination-item pagination-last disabled']").text == "Last":
            break       # Break the loop if the element is present
    except:
        # if the above element is not present then click on ">" that is the next button
        driver.find_element_by_xpath("//i[@class='core-icon icon-core-chevron-right']").click()
        time.sleep(3)

In [31]:
for i in range(len(distance)):
    # Splitting the each distance string to get it only till km and discarding rest string
    distance[i]=distance[i].split("- ")[1].split(" from")[0] 

In [32]:
for i in range(len(prices)): # Running for loop over prices list
    if i%2==0:
        privates_from_price.append(prices[i])   # Storing Privates From price
    else:
        dorms_from_price.append(prices[i])      # Storing Dorms From price

In [33]:
# Cleaning the Privates From price list
for i in range(len(privates_from_price)):
    if privates_from_price[i]!="No Privates Available":
        privates_from_price[i]=privates_from_price[i].split("\n")[1]  # Storing only the price and discarding the rest string
    else:
        continue

In [34]:
# Cleaning the Dorms From price list
for i in range(len(dorms_from_price)):
    if dorms_from_price[i]!="No Dorms Available":
        dorms_from_price[i]=dorms_from_price[i].split("\n")[1]  # Storing only the price and discarding the rest string
    else:
        continue

In [35]:
# Running for loop over each of the hostel page 
for url in urls:
    driver.get(url) # opening the page 
    
    # Scraping the Ratings of the hostel
    try:
        r=driver.find_element_by_xpath("//div[@class='score orange big']")
        ratings.append(r.text)
    except NoSuchElementException:
        ratings.append("-")
     
    # Scraping the Overall Reviews
    try:
        ovr=driver.find_element_by_xpath("//div[@class='flex-20']//div[1]//div[2]//div[1]")
        overall_reviews.append(ovr.text)
    except NoSuchElementException:
        overall_reviews.append("-")
     
    # Scraping the Total Reviews
    try:
        tr=driver.find_element_by_xpath("//div[@class='flex-20']//div[1]//div[2]//div[2]")
        total_reviews.append(tr.text)
    except NoSuchElementException:
        total_reviews.append("-")
      
    # Scraping the Facilities
    try:
        if driver.find_element_by_xpath("//ul[@class='facility-sections']//li[1]//h2").text == "Free":
            f=driver.find_element_by_xpath("//ul[@class='facility-sections']//li[1]//ul[1]")
            facility.append(f.text)
        elif driver.find_element_by_xpath("//ul[@class='facility-sections']//li[1]//h2").text == "General":
            g=driver.find_element_by_xpath("//ul[@class='facility-sections']//li[2]//ul[1]")
            facility.append(g.text)
        elif driver.find_element_by_xpath("//ul[@class='facility-sections']//li[1]//h2").text == "Services":
            s=driver.find_element_by_xpath("//ul[@class='facility-sections']//li[3]//ul[1]")
            facility.append(s.text)
        elif driver.find_element_by_xpath("//ul[@class='facility-sections']//li[1]//h2").text == "Food & Drink":
            d=driver.find_element_by_xpath("//ul[@class='facility-sections']//li[4]//ul[1]")
            facility.append(d.text)
        elif driver.find_element_by_xpath("//ul[@class='facility-sections']//li[1]//h2").text == "Entertainment":
            e=driver.find_element_by_xpath("//ul[@class='facility-sections']//li[5]//ul[1]")
            facility.append(e.text)
        facilities.append(facility)
    except NoSuchElementException:
        facilities.append("-")
       
    # Scraping the Property Description
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        prds=driver.find_element_by_xpath("//div[@class='collapsible']//div[2]")
        property_desc.append(prds.text)
    except NoSuchElementException:
        property_desc.append("-")

In [36]:
for i in range(len(total_reviews)):
    total_reviews[i]=total_reviews[i].split(" Total")[0]   # Storing only the count of Total Reviews and discarding the string

In [37]:
# Creating the Dataframe for the scraped data of Hostel
Hostel=pd.DataFrame({"Hostel Name":hostel_name,
                "Distance from City Centre":distance,
                "Ratings":ratings,
                "Total Reviews":total_reviews,
                "Overall Reviews":overall_reviews,
                 "Privates from Price":privates_from_price,
                "Dorms from Price":dorms_from_price,
                "Facilities":facilities,
                "Property Description":property_desc})

# Printing the DataFrame
Hostel

,Hostel Name,Distance from City Centre,Ratings,Total Reviews,Overall Reviews,Privates from Price,Dorms from Price,Facilities,Property Description
0,London Waterloo Hostel,0.7km,7.7,2408,Very Good,No Privates Available,Rs1520,[Free Breakfast Linen Included Free City Maps ...,COVID-19 Policy Update\n\nIn response to Coron...
1,Wombat's The City Hostel London,3.6km,9.2,13140,Superb,Rs7845,Rs1818,[Free Breakfast Linen Included Free City Maps ...,Wombat's The City Hostel London is definitely ...
2,Prime Backpackers Angel,3.6km,10,513,Superb,Rs6039,Rs2397,[Free Breakfast Linen Included Free City Maps ...,The hostel is housed in a historic building in...
3,The Walrus Hostel,0.2km,8.4,3646,Fabulous,No Privates Available,Rs1277,[Free Breakfast Linen Included Free City Maps ...,If you're looking for a spot that's as central...
4,St Christopher's Village,1.8km,8.9,10827,Fabulous,Rs3715,Rs1256,[Free Breakfast Linen Included Free City Maps ...,Hostelworld visitors have said St Christopher'...
...,...,...,...,...,...,...,...,...,...
80,The Dover,1.9km,-,-,-,Rs6788,No Dorms Available,[Free Breakfast Linen Included Free City Maps ...,"With friendly staff, excellent service and an ..."
81,Park Hotel Essex,24.1km,-,-,-,Rs4154,No Dorms Available,[Free Breakfast Linen Included Free City Maps ...,This Hotel is the right choice for visitors wh...
82,Cranbrook Hotel,14.8km,-,-,-,Rs3951,No Dorms Available,[Free Breakfast Linen Included Free City Maps ...,We are located about twenty minutes by tube fr...
83,St. Athans,2.9km,-,-,-,Rs4083,No Dorms Available,[Free Breakfast Linen Included Free City Maps ...,"The St Athans Hotel is a proudly simple, famil..."


# Thank you!